<a href="https://colab.research.google.com/github/yefry08/1791/blob/main/Makers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install gradio plotly pandas numpy spacy streamlit
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import re
import json
from typing import Dict, List, Tuple, Any
from difflib import SequenceMatcher
from collections import defaultdict, Counter
import statistics

class MakersTechChatbot:
    def __init__(self):
        self.inventory = self.load_inventory()
        self.synonyms = self.load_synonyms()
        self.intent_patterns = self.load_intent_patterns()

    def load_inventory(self) -> List[Dict]:
        """Load 10 sample computer products"""
        return [
            {
                "id": 1,
                "brand": "Apple",
                "model": "MacBook Pro 14-inch",
                "type": "laptop",
                "price": 1999,
                "stock_quantity": 5,
                "features": {
                    "ram": "16GB",
                    "processor": "M2 Pro",
                    "storage": "512GB SSD",
                    "screen_size": "14-inch",
                    "graphics": "Integrated"
                }
            },
            {
                "id": 2,
                "brand": "Dell",
                "model": "XPS 13",
                "type": "laptop",
                "price": 1299,
                "stock_quantity": 8,
                "features": {
                    "ram": "16GB",
                    "processor": "Intel Core i7",
                    "storage": "256GB SSD",
                    "screen_size": "13.3-inch",
                    "graphics": "Intel Iris Xe"
                }
            },
            {
                "id": 3,
                "brand": "HP",
                "model": "Pavilion Gaming Desktop",
                "type": "desktop",
                "price": 899,
                "stock_quantity": 3,
                "features": {
                    "ram": "8GB",
                    "processor": "AMD Ryzen 5",
                    "storage": "1TB HDD + 256GB SSD",
                    "screen_size": "N/A",
                    "graphics": "NVIDIA GTX 1650"
                }
            },
            {
                "id": 4,
                "brand": "Apple",
                "model": "iMac 24-inch",
                "type": "desktop",
                "price": 1499,
                "stock_quantity": 4,
                "features": {
                    "ram": "8GB",
                    "processor": "M1",
                    "storage": "256GB SSD",
                    "screen_size": "24-inch",
                    "graphics": "Integrated"
                }
            },
            {
                "id": 5,
                "brand": "Dell",
                "model": "Inspiron 15 3000",
                "type": "laptop",
                "price": 599,
                "stock_quantity": 12,
                "features": {
                    "ram": "8GB",
                    "processor": "Intel Core i5",
                    "storage": "256GB SSD",
                    "screen_size": "15.6-inch",
                    "graphics": "Intel UHD"
                }
            },
            {
                "id": 6,
                "brand": "HP",
                "model": "Envy x360",
                "type": "laptop",
                "price": 849,
                "stock_quantity": 6,
                "features": {
                    "ram": "16GB",
                    "processor": "AMD Ryzen 7",
                    "storage": "512GB SSD",
                    "screen_size": "15.6-inch",
                    "graphics": "AMD Radeon"
                }
            },
            {
                "id": 7,
                "brand": "ASUS",
                "model": "ROG Strix Desktop",
                "type": "desktop",
                "price": 2199,
                "stock_quantity": 2,
                "features": {
                    "ram": "32GB",
                    "processor": "Intel Core i9",
                    "storage": "1TB SSD",
                    "screen_size": "N/A",
                    "graphics": "NVIDIA RTX 3080"
                }
            },
            {
                "id": 8,
                "brand": "Lenovo",
                "model": "ThinkPad E15",
                "type": "laptop",
                "price": 999,
                "stock_quantity": 7,
                "features": {
                    "ram": "16GB",
                    "processor": "Intel Core i7",
                    "storage": "512GB SSD",
                    "screen_size": "15.6-inch",
                    "graphics": "Intel Iris Xe"
                }
            },
            {
                "id": 9,
                "brand": "HP",
                "model": "Omen Desktop",
                "type": "desktop",
                "price": 1599,
                "stock_quantity": 3,
                "features": {
                    "ram": "16GB",
                    "processor": "Intel Core i7",
                    "storage": "1TB SSD",
                    "screen_size": "N/A",
                    "graphics": "NVIDIA RTX 3060"
                }
            },
            {
                "id": 10,
                "brand": "Acer",
                "model": "Aspire 5",
                "type": "laptop",
                "price": 699,
                "stock_quantity": 10,
                "features": {
                    "ram": "8GB",
                    "processor": "AMD Ryzen 5",
                    "storage": "256GB SSD",
                    "screen_size": "15.6-inch",
                    "graphics": "AMD Radeon"
                }
            }
        ]

    def load_synonyms(self) -> Dict[str, List[str]]:
        """Load synonym mappings for better query understanding"""
        return {
            "computer": ["pc", "machine", "system", "device", "unit"],
            "laptop": ["notebook", "portable", "mobile computer"],
            "desktop": ["tower", "workstation", "pc tower"],
            "price": ["cost", "pricing", "expense", "fee", "amount"],
            "features": ["specs", "specifications", "details", "characteristics"],
            "memory": ["ram", "memory"],
            "storage": ["hard drive", "hdd", "ssd", "disk space"],
            "processor": ["cpu", "chip", "core"],
            "buy": ["purchase", "get", "acquire", "order"],
            "available": ["in stock", "stock", "inventory"],
            "cheap": ["affordable", "budget", "low-cost", "inexpensive"],
            "expensive": ["premium", "high-end", "costly"]
        }

    def load_intent_patterns(self) -> Dict[str, List[str]]:
        """Define patterns for intent recognition"""
        return {
            "inventory_count": [
                r"how many.*computers?.*available",
                r"how many.*in stock",
                r"what.*inventory",
                r"total.*computers"
            ],
            "product_info": [
                r"tell me.*about.*",
                r"what.*know.*about",
                r"details.*about",
                r"information.*about"
            ],
            "price_query": [
                r"what.*price.*",
                r"how much.*cost",
                r"price.*of",
                r"cheapest.*",
                r"most expensive"
            ],
            "feature_search": [
                r".*with.*ram",
                r".*with.*gb",
                r".*processor.*",
                r".*graphics.*"
            ],
            "brand_search": [
                r".*apple.*",
                r".*dell.*",
                r".*hp.*",
                r".*lenovo.*",
                r".*asus.*",
                r".*acer.*"
            ],
            "purchase_intent": [
                r".*buy.*",
                r".*purchase.*",
                r".*order.*",
                r".*get.*"
            ]
        }

    def normalize_text(self, text: str) -> str:
        """Normalize input text for better matching"""
        text = text.lower().strip()
        # Replace synonyms
        for key, synonyms in self.synonyms.items():
            for synonym in synonyms:
                text = re.sub(r'\b' + synonym + r'\b', key, text)
        return text

    def identify_intent(self, user_input: str) -> str:
        """Identify user intent from input text"""
        normalized_input = self.normalize_text(user_input)

        for intent, patterns in self.intent_patterns.items():
            for pattern in patterns:
                if re.search(pattern, normalized_input):
                    return intent

        return "general_inquiry"

    def extract_brand(self, text: str) -> str:
        """Extract brand name from user input"""
        text = text.lower()
        brands = ["apple", "dell", "hp", "lenovo", "asus", "acer"]
        for brand in brands:
            if brand in text:
                return brand.title()
        return None

    def extract_price_range(self, text: str) -> Tuple[int, int]:
        """Extract price range from user input"""
        # Look for price mentions
        price_patterns = [
            r'under \$?(\d+)',
            r'below \$?(\d+)',
            r'less than \$?(\d+)',
            r'between \$?(\d+).*\$?(\d+)',
            r'from \$?(\d+).*to \$?(\d+)'
        ]

        for pattern in price_patterns:
            match = re.search(pattern, text.lower())
            if match:
                if len(match.groups()) == 1:
                    return (0, int(match.group(1)))
                elif len(match.groups()) == 2:
                    return (int(match.group(1)), int(match.group(2)))

        return None

    def extract_ram_requirement(self, text: str) -> str:
        """Extract RAM requirement from user input"""
        ram_pattern = r'(\d+)\s*gb.*ram|ram.*(\d+)\s*gb'
        match = re.search(ram_pattern, text.lower())
        if match:
            ram_size = match.group(1) or match.group(2)
            return f"{ram_size}GB"
        return None

    def fuzzy_match_products(self, query: str, threshold: float = 0.3) -> List[Dict]:
        """Find products using fuzzy string matching"""
        matches = []
        query_lower = query.lower()

        for product in self.inventory:
            score = 0

            # Check brand match
            if product['brand'].lower() in query_lower:
                score += 0.4

            # Check model match
            model_similarity = SequenceMatcher(None, query_lower, product['model'].lower()).ratio()
            score += model_similarity * 0.3

            # Check feature matches
            features_text = ' '.join([str(v) for v in product['features'].values()])
            feature_similarity = SequenceMatcher(None, query_lower, features_text.lower()).ratio()
            score += feature_similarity * 0.3

            if score >= threshold:
                matches.append((product, score))

        # Sort by score descending
        matches.sort(key=lambda x: x[1], reverse=True)
        return [match[0] for match in matches]

    def filter_products(self, brand=None, price_range=None, ram_requirement=None, product_type=None) -> List[Dict]:
        """Filter products based on criteria"""
        filtered = self.inventory

        if brand:
            filtered = [p for p in filtered if p['brand'].lower() == brand.lower()]

        if price_range:
            min_price, max_price = price_range
            filtered = [p for p in filtered if min_price <= p['price'] <= max_price]

        if ram_requirement:
            filtered = [p for p in filtered if ram_requirement in p['features']['ram']]

        if product_type:
            filtered = [p for p in filtered if p['type'] == product_type.lower()]

        return filtered

    def format_product_info(self, product: Dict) -> str:
        """Format product information for display"""
        features = product['features']
        return (f"**{product['brand']} {product['model']}** - ${product['price']:,}\n"
                f"• RAM: {features['ram']}\n"
                f"• Processor: {features['processor']}\n"
                f"• Storage: {features['storage']}\n"
                f"• Screen: {features.get('screen_size', 'N/A')}\n"
                f"• Graphics: {features['graphics']}\n"
                f"• Stock: {product['stock_quantity']} available")

    def process_user_input(self, user_input: str) -> str:
        """Main function to process user input and return response"""
        intent = self.identify_intent(user_input)

        if intent == "inventory_count":
            return self.handle_inventory_count(user_input)
        elif intent == "product_info":
            return self.handle_product_info(user_input)
        elif intent == "price_query":
            return self.handle_price_query(user_input)
        elif intent == "feature_search":
            return self.handle_feature_search(user_input)
        elif intent == "brand_search":
            return self.handle_brand_search(user_input)
        elif intent == "purchase_intent":
            return self.handle_purchase_intent(user_input)
        else:
            return self.handle_general_inquiry(user_input)

    def handle_inventory_count(self, user_input: str) -> str:
        """Handle inventory count queries"""
        brand = self.extract_brand(user_input)

        if brand:
            count = len([p for p in self.inventory if p['brand'] == brand])
            if count > 0:
                return f"We currently have {count} {brand} computers in stock! Would you like to see the available models?"
            else:
                return f"I'm sorry, we don't currently have any {brand} computers in stock. Can I help you find alternatives from other brands?"
        else:
            total_count = len(self.inventory)
            laptop_count = len([p for p in self.inventory if p['type'] == 'laptop'])
            desktop_count = len([p for p in self.inventory if p['type'] == 'desktop'])

            return (f"We have a total of {total_count} computers available: "
                   f"{laptop_count} laptops and {desktop_count} desktops. "
                   f"Would you like to explore our selection?")

    def handle_product_info(self, user_input: str) -> str:
        """Handle product information requests"""
        brand = self.extract_brand(user_input)

        if brand:
            products = [p for p in self.inventory if p['brand'] == brand]
            if products:
                if len(products) == 1:
                    return f"Here's information about our {brand} computer:\n\n{self.format_product_info(products[0])}\n\nWould you like to know more or are you interested in purchasing?"
                else:
                    response = f"We have {len(products)} {brand} computers available:\n\n"
                    for product in products[:3]:  # Show top 3
                        response += f"• {product['model']} - ${product['price']:,}\n"
                    response += f"\nWould you like detailed information about any specific model?"
                    return response

        # Fallback to fuzzy matching
        matches = self.fuzzy_match_products(user_input)
        if matches:
            return f"I found this computer that might interest you:\n\n{self.format_product_info(matches[0])}\n\nIs this what you were looking for?"

        return "I'd be happy to help you find the right computer! Could you tell me more about what you're looking for? For example, do you prefer a specific brand or have any feature requirements?"

    def handle_price_query(self, user_input: str) -> str:
        """Handle price-related queries"""
        if "cheap" in user_input.lower() or "budget" in user_input.lower():
            cheapest = min(self.inventory, key=lambda x: x['price'])
            return (f"Our most affordable computer is the **{cheapest['brand']} {cheapest['model']}** "
                   f"at just ${cheapest['price']:,}! It offers great value with {cheapest['features']['ram']} RAM "
                   f"and {cheapest['features']['processor']} processor. Would you like more details?")

        elif "expensive" in user_input.lower() or "premium" in user_input.lower():
            most_expensive = max(self.inventory, key=lambda x: x['price'])
            return (f"Our premium computer is the **{most_expensive['brand']} {most_expensive['model']}** "
                   f"at ${most_expensive['price']:,}. It features top-tier specs including "
                   f"{most_expensive['features']['ram']} RAM and {most_expensive['features']['processor']} processor. "
                   f"Perfect for demanding tasks!")

        # Extract specific brand for price query
        brand = self.extract_brand(user_input)
        if brand:
            brand_products = [p for p in self.inventory if p['brand'] == brand]
            if brand_products:
                cheapest_brand = min(brand_products, key=lambda x: x['price'])
                return (f"The most affordable {brand} computer we have is the **{cheapest_brand['model']}** "
                       f"starting at ${cheapest_brand['price']:,}. It's a great choice with excellent build quality!")

        return "I can help you find computers in your price range! What's your budget, or would you like to see our most affordable options?"

    def handle_feature_search(self, user_input: str) -> str:
        """Handle feature-based searches"""
        ram_req = self.extract_ram_requirement(user_input)
        brand = self.extract_brand(user_input)

        filters = {}
        if brand:
            filters['brand'] = brand
        if ram_req:
            filters['ram_requirement'] = ram_req

        matching_products = self.filter_products(**filters)

        if matching_products:
            if len(matching_products) == 1:
                product = matching_products[0]
                return (f"Perfect! I found exactly what you're looking for:\n\n{self.format_product_info(product)}\n\n"
                       f"This computer meets your requirements perfectly. Would you like to proceed with this option?")
            else:
                response = f"Great! I found {len(matching_products)} computers that match your requirements:\n\n"
                for product in matching_products[:3]:
                    response += f"• **{product['brand']} {product['model']}** - ${product['price']:,} ({product['features']['ram']} RAM)\n"
                if len(matching_products) > 3:
                    response += f"...and {len(matching_products) - 3} more options.\n"
                response += "\nWould you like detailed information about any of these?"
                return response
        else:
            return (f"I don't currently have computers that exactly match those specifications. "
                   f"However, I can show you similar options that might work for you. "
                   f"What's most important to you - the RAM size, brand, or another feature?")

    def handle_brand_search(self, user_input: str) -> str:
        """Handle brand-specific searches"""
        brand = self.extract_brand(user_input)
        if brand:
            brand_products = [p for p in self.inventory if p['brand'] == brand]
            if brand_products:
                response = f"Excellent choice! We have {len(brand_products)} {brand} computers available:\n\n"
                for product in brand_products:
                    response += f"• **{product['model']}** - ${product['price']:,} ({product['type']})\n"
                response += f"\n{brand} is known for quality and reliability. Which model interests you most?"
                return response

        return "I'd be happy to help you explore different brands! We carry Apple, Dell, HP, Lenovo, ASUS, and Acer. Which brand interests you?"

    def handle_purchase_intent(self, user_input: str) -> str:
        """Handle purchase-related inquiries"""
        # Try to identify specific product mentioned
        matches = self.fuzzy_match_products(user_input)

        if matches:
            product = matches[0]
            return (f"Excellent choice! The **{product['brand']} {product['model']}** is a fantastic computer. "
                   f"It's available for ${product['price']:,} and we have {product['stock_quantity']} in stock. "
                   f"To complete your purchase, I can connect you with our sales team. "
                   f"Would you like me to arrange that for you?")
        else:
            return ("I'd love to help you make a purchase! To recommend the perfect computer for you, "
                   "could you tell me what you'll primarily use it for? Gaming, work, general use? "
                   "And do you have a preferred budget range?")

    def handle_general_inquiry(self, user_input: str) -> str:
        """Handle general inquiries"""
        return ("Hi! I'm here to help you find the perfect computer at Makers Tech. "
               "We have a great selection of laptops and desktops from top brands like Apple, Dell, HP, and more. "
               "What can I help you with today? You can ask about specific brands, features, prices, or just tell me what you need a computer for!")

    def recommend_product(self, preferences: Dict[str, Any]) -> Dict[str, List[Dict]]:
        """Recommend products based on user preferences"""
        recommendations = {
            "highly_recommended": [],
            "recommended": [],
            "not_recommended": []
        }

        for product in self.inventory:
            score = 0

            # Brand preference
            if preferences.get('brand') and product['brand'].lower() == preferences['brand'].lower():
                score += 3

            # Price range preference
            if preferences.get('max_budget'):
                if product['price'] <= preferences['max_budget']:
                    score += 2
                else:
                    score -= 2

            # RAM preference
            if preferences.get('min_ram'):
                product_ram = int(product['features']['ram'].replace('GB', ''))
                if product_ram >= preferences['min_ram']:
                    score += 2
                else:
                    score -= 1

            # Type preference (laptop vs desktop)
            if preferences.get('type') and product['type'] == preferences['type'].lower():
                score += 2

            # Use case scoring
            use_case = preferences.get('use_case', '').lower()
            if 'gaming' in use_case:
                if 'nvidia' in product['features']['graphics'].lower() or 'amd radeon' in product['features']['graphics'].lower():
                    score += 2
                if 'integrated' in product['features']['graphics'].lower():
                    score -= 1

            if 'work' in use_case or 'business' in use_case:
                if product['brand'] in ['Dell', 'Lenovo', 'HP']:
                    score += 1

            # Stock availability
            if product['stock_quantity'] > 0:
                score += 1
            else:
                score -= 3

            # Categorize based on score
            if score >= 5:
                recommendations["highly_recommended"].append(product)
            elif score >= 2:
                recommendations["recommended"].append(product)
            else:
                recommendations["not_recommended"].append(product)

        # Sort each category by price (ascending)
        for category in recommendations:
            recommendations[category].sort(key=lambda x: x['price'])

        return recommendations

    def get_dashboard_data(self) -> Dict[str, Any]:
        """Return admin dashboard metrics"""
        # Products by brand
        brand_counts = Counter(product['brand'] for product in self.inventory)

        # Stock levels
        total_stock = sum(product['stock_quantity'] for product in self.inventory)
        low_stock_products = [p for p in self.inventory if p['stock_quantity'] <= 3]

        # Price distribution
        prices = [product['price'] for product in self.inventory]
        price_stats = {
            'min': min(prices),
            'max': max(prices),
            'average': round(statistics.mean(prices), 2),
            'median': statistics.median(prices)
        }

        # Type distribution
        type_counts = Counter(product['type'] for product in self.inventory)

        return {
            'total_products': len(self.inventory),
            'total_stock': total_stock,
            'products_by_brand': dict(brand_counts),
            'products_by_type': dict(type_counts),
            'low_stock_alerts': len(low_stock_products),
            'low_stock_products': [f"{p['brand']} {p['model']}" for p in low_stock_products],
            'price_statistics': price_stats,
            'brands_available': list(brand_counts.keys())
        }

# Example usage and testing
if __name__ == "__main__":
    # Initialize the chatbot
    chatbot = MakersTechChatbot()

    # Test various queries
    test_queries = [
        "How many computers do you have available?",
        "Tell me about Apple computers",
        "What's the price of the cheapest Dell laptop?",
        "Do you have any HP computers with 16GB RAM?",
        "I want to buy a gaming computer",
        "Show me budget laptops under $800",
        "What are the specs of the MacBook Pro?"
    ]

    print("=== Makers Tech Chatbot Test ===\n")

    for query in test_queries:
        print(f"User: {query}")
        response = chatbot.process_user_input(query)
        print(f"Bot: {response}\n")
        print("-" * 50)

    # Test recommendation system
    print("\n=== Recommendation Test ===")
    preferences = {
        'brand': 'HP',
        'max_budget': 1000,
        'min_ram': 16,
        'type': 'laptop',
        'use_case': 'work and productivity'
    }

    recommendations = chatbot.recommend_product(preferences)

    print("Highly Recommended:")
    for product in recommendations['highly_recommended']:
        print(f"- {product['brand']} {product['model']} (${product['price']})")

    print("\nRecommended:")
    for product in recommendations['recommended']:
        print(f"- {product['brand']} {product['model']} (${product['price']})")

    # Test dashboard data
    print("\n=== Dashboard Data ===")
    dashboard = chatbot.get_dashboard_data()
    print(f"Total Products: {dashboard['total_products']}")
    print(f"Total Stock: {dashboard['total_stock']}")
    print(f"Brands: {dashboard['brands_available']}")
    print(f"Low Stock Alerts: {dashboard['low_stock_alerts']}")
    print(f"Price Range: ${dashboard['price_statistics']['min']} - ${dashboard['price_statistics']['max']}")

import gradio as gr
import json
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd

# Import the chatbot backend (assuming it's in the same directory)
# from makers_tech_chatbot import MakersTechChatbot

# For demo purposes, I'll include a simplified version of the chatbot class
import re
import statistics
from typing import Dict, List, Tuple, Any
from difflib import SequenceMatcher
from collections import defaultdict, Counter

class MakersTechChatbot:
    def __init__(self):
        self.inventory = self.load_inventory()
        self.synonyms = self.load_synonyms()
        self.intent_patterns = self.load_intent_patterns()

    def load_inventory(self) -> List[Dict]:
        """Load sample computer products with emojis"""
        return [
            {
                "id": 1, "brand": "Apple", "model": "MacBook Pro 14-inch", "type": "laptop", "price": 1999,
                "stock_quantity": 5, "emoji": "💻", "category": "Computers",
                "features": {"ram": "16GB", "processor": "M2 Pro", "storage": "512GB SSD", "screen_size": "14-inch", "graphics": "Integrated"}
            },
            {
                "id": 2, "brand": "Dell", "model": "XPS 13", "type": "laptop", "price": 1299,
                "stock_quantity": 1, "emoji": "💻", "category": "Computers",
                "features": {"ram": "16GB", "processor": "Intel Core i7", "storage": "256GB SSD", "screen_size": "13.3-inch", "graphics": "Intel Iris Xe"}
            },
            {
                "id": 3, "brand": "HP", "model": "Pavilion Gaming Desktop", "type": "desktop", "price": 899,
                "stock_quantity": 2, "emoji": "🖥️", "category": "Computers",
                "features": {"ram": "8GB", "processor": "AMD Ryzen 5", "storage": "1TB HDD + 256GB SSD", "screen_size": "N/A", "graphics": "NVIDIA GTX 1650"}
            },
            {
                "id": 4, "brand": "Apple", "model": "iPhone 15 Pro", "type": "phone", "price": 999,
                "stock_quantity": 5, "emoji": "📱", "category": "Phones",
                "features": {"storage": "128GB", "camera": "48MP", "display": "6.1-inch", "chip": "A17 Pro"}
            },
            {
                "id": 5, "brand": "Samsung", "model": "Galaxy S24 Ultra", "type": "phone", "price": 1199,
                "stock_quantity": 3, "emoji": "📱", "category": "Phones",
                "features": {"storage": "256GB", "camera": "200MP", "display": "6.8-inch", "chip": "Snapdragon 8 Gen 3"}
            },
            {
                "id": 6, "brand": "Apple", "model": "AirPods Pro (2nd Gen)", "type": "accessory", "price": 249,
                "stock_quantity": 8, "emoji": "🎧", "category": "Accessories",
                "features": {"type": "Wireless Earbuds", "noise_cancellation": "Active", "battery": "6 hours"}
            },
            {
                "id": 7, "brand": "Logitech", "model": "MX Master 3S", "type": "accessory", "price": 99,
                "stock_quantity": 12, "emoji": "🖱️", "category": "Accessories",
                "features": {"type": "Wireless Mouse", "connectivity": "Bluetooth/USB-C", "battery": "70 days"}
            }
        ]

    def load_synonyms(self) -> Dict[str, List[str]]:
        return {
            "computer": ["pc", "laptop", "desktop", "machine"],
            "phone": ["smartphone", "mobile", "cell phone"],
            "price": ["cost", "pricing", "expense"],
            "available": ["in stock", "inventory"],
        }

    def load_intent_patterns(self) -> Dict[str, List[str]]:
        return {
            "inventory_count": [r"how many.*available", r"total.*products"],
            "product_info": [r"tell me.*about", r"information.*about"],
            "price_query": [r"what.*price", r"how much.*cost"],
        }

    def process_user_input(self, user_input: str) -> str:
        """Simplified processing for demo"""
        user_input_lower = user_input.lower()

        if "how many" in user_input_lower and "computer" in user_input_lower:
            computers = [p for p in self.inventory if p['category'] == 'Computers']
            return f"We currently have {len(computers)} computers in stock! 💻 Would you like to see our available models?"

        elif "dell laptop" in user_input_lower:
            dell_laptops = [p for p in self.inventory if p['brand'] == 'Dell' and p['type'] == 'laptop']
            if dell_laptops:
                laptop = dell_laptops[0]
                return f"Great choice! The **{laptop['model']}** is available for ${laptop['price']:,} with {laptop['features']['ram']} RAM and {laptop['features']['processor']} processor. We have {laptop['stock_quantity']} in stock. Would you like more details? 💻"

        elif "apple" in user_input_lower:
            apple_products = [p for p in self.inventory if p['brand'] == 'Apple']
            response = f"We have {len(apple_products)} Apple products available:\n\n"
            for product in apple_products:
                response += f"• {product['emoji']} **{product['model']}** - ${product['price']:,}\n"
            response += "\nWhich Apple product interests you most? 🍎"
            return response

        elif "gaming" in user_input_lower:
            return "Perfect! For gaming, I'd recommend our **HP Pavilion Gaming Desktop** 🎮 It's equipped with AMD Ryzen 5 processor and NVIDIA GTX 1650 graphics for excellent gaming performance at $899. We have 2 units in stock!"

        return "Hello! I'm your Makers Tech Assistant 🤖 I can help you with our inventory, product information, and recommendations. What can I help you find today?"

    def get_dashboard_data(self) -> Dict[str, Any]:
        """Return dashboard metrics"""
        # Calculate metrics
        total_products = len(self.inventory)
        total_stock = sum(p['stock_quantity'] for p in self.inventory)
        low_stock_items = len([p for p in self.inventory if p['stock_quantity'] <= 3])
        categories = len(set(p['category'] for p in self.inventory))

        # Stock by category
        category_stock = {}
        for product in self.inventory:
            category = product['category']
            category_stock[category] = category_stock.get(category, 0) + product['stock_quantity']

        # Low stock alerts
        low_stock_products = [p for p in self.inventory if p['stock_quantity'] <= 3]

        # Top value inventory
        top_value = sorted(self.inventory, key=lambda x: x['price'] * x['stock_quantity'], reverse=True)[:5]

        return {
            'total_products': total_products,
            'total_stock': total_stock,
            'low_stock_items': low_stock_items,
            'categories': categories,
            'category_stock': category_stock,
            'low_stock_products': low_stock_products,
            'top_value_inventory': top_value
        }

# Initialize chatbot
chatbot = MakersTechChatbot()

def create_stock_chart():
    """Create stock by category chart"""
    dashboard_data = chatbot.get_dashboard_data()
    categories = list(dashboard_data['category_stock'].keys())
    stock_counts = list(dashboard_data['category_stock'].values())

    fig = go.Figure(data=[
        go.Bar(x=categories, y=stock_counts,
               marker_color='#6366f1',
               text=stock_counts,
               textposition='auto')
    ])

    fig.update_layout(
        title="Stock by Category",
        xaxis_title="Categories",
        yaxis_title="Stock Count",
        height=300,
        margin=dict(l=20, r=20, t=40, b=20),
        font=dict(size=12)
    )

    return fig

def create_stock_distribution_chart():
    """Create stock level distribution pie chart"""
    dashboard_data = chatbot.get_dashboard_data()

    in_stock = len([p for p in chatbot.inventory if p['stock_quantity'] > 3])
    low_stock = len([p for p in chatbot.inventory if p['stock_quantity'] <= 3 and p['stock_quantity'] > 0])
    out_of_stock = len([p for p in chatbot.inventory if p['stock_quantity'] == 0])

    fig = go.Figure(data=[go.Pie(
        labels=['In Stock', 'Low Stock', 'Out of Stock'],
        values=[in_stock, low_stock, out_of_stock],
        marker=dict(colors=['#10b981', '#f59e0b', '#ef4444']),
        hole=.3
    )])

    fig.update_layout(
        title="Stock Level Distribution",
        height=300,
        margin=dict(l=20, r=20, t=40, b=20),
        font=dict(size=12)
    )

    return fig

def format_chat_response(message, is_user=True):
    """Format chat messages"""
    if is_user:
        return f'<div style="text-align: right; margin: 10px 0;"><div style="background: #6366f1; color: white; padding: 10px 15px; border-radius: 18px; display: inline-block; max-width: 70%;">{message}</div></div>'
    else:
        return f'<div style="text-align: left; margin: 10px 0;"><div style="background: #f3f4f6; color: #374151; padding: 10px 15px; border-radius: 18px; display: inline-block; max-width: 70%;">{message}</div></div>'

def chat_interface(message, history):
    """Handle chat interface"""
    if not message.strip():
        return history, ""

    # Get bot response
    bot_response = chatbot.process_user_input(message)

    # Update history with proper message format
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": bot_response})

    return history, ""

def get_products_display():
    """Get products for display"""
    products_html = ""
    for product in chatbot.inventory:
        stock_status = "In Stock" if product['stock_quantity'] > 3 else "Low Stock" if product['stock_quantity'] > 0 else "Out of Stock"
        stock_color = "#10b981" if product['stock_quantity'] > 3 else "#f59e0b" if product['stock_quantity'] > 0 else "#ef4444"

        products_html += f"""
        <div style="border: 1px solid #e5e7eb; border-radius: 8px; padding: 15px; margin: 10px 0; background: white;">
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <span style="font-size: 24px; margin-right: 10px;">{product['emoji']}</span>
                <div>
                    <h3 style="margin: 0; color: #374151; font-size: 16px;">{product['brand']} {product['model']}</h3>
                    <p style="margin: 5px 0 0 0; color: #3333333; font-size: 14px;">{product['category']}</p>
                </div>
            </div>
            <div style="display: flex; justify-content: space-between; align-items: center;">
                <span style="font-size: 18px; font-weight: bold; color: #6366f1;">${product['price']:,}</span>
                <span style="color: {stock_color}; font-size: 12px; padding: 4px 8px; background: {stock_color}20; border-radius: 12px;">{product['stock_quantity']} in stock</span>
            </div>
            <button style="width: 100%; margin-top: 10px; background: #6366f1; color: white; border: none; padding: 8px; border-radius: 6px; cursor: pointer;">🛒 Add to Cart</button>
        </div>
        """
    return products_html

def get_dashboard_display():
    """Get dashboard metrics display"""
    dashboard_data = chatbot.get_dashboard_data()

    # Metrics cards
    metrics_html = f"""
    <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 20px; margin-bottom: 30px;">
        <div style="background: white; border-radius: 8px; padding: 20px; border: 1px solid #e5e7eb;">
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <span style="font-size: 24px; margin-right: 10px;">📦</span>
                <span style="color: #6b7280; font-size: 14px;">Total Products</span>
            </div>
            <div style="font-size: 32px; font-weight: bold; color: #374151;">{dashboard_data['total_products']}</div>
        </div>

        <div style="background: white; border-radius: 8px; padding: 20px; border: 1px solid #e5e7eb;">
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <span style="font-size: 24px; margin-right: 10px;">📈</span>
                <span style="color: #6b7280; font-size: 14px;">Total Stock</span>
            </div>
            <div style="font-size: 32px; font-weight: bold; color: #374151;">{dashboard_data['total_stock']}</div>
        </div>

        <div style="background: white; border-radius: 8px; padding: 20px; border: 1px solid #e5e7eb;">
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <span style="font-size: 24px; margin-right: 10px;">⚠️</span>
                <span style="color: #6b7280; font-size: 14px;">Low Stock Items</span>
            </div>
            <div style="font-size: 32px; font-weight: bold; color: #374151;">{dashboard_data['low_stock_items']}</div>
        </div>

        <div style="background: white; border-radius: 8px; padding: 20px; border: 1px solid #e5e7eb;">
            <div style="display: flex; align-items: center; margin-bottom: 10px;">
                <span style="font-size: 24px; margin-right: 10px;">🏷️</span>
                <span style="color: #6b7280; font-size: 14px;">Categories</span>
            </div>
            <div style="font-size: 32px; font-weight: bold; color: #374151;">{dashboard_data['categories']}</div>
        </div>
    </div>
    """

    # Low stock alerts
    low_stock_html = """
    <div style="background: white; border-radius: 8px; padding: 20px; border: 1px solid #e5e7eb; margin-bottom: 20px;">
        <h3 style="color: #374151; margin-bottom: 15px; display: flex; align-items: center;">
            <span style="margin-right: 10px;">⚠️</span> Low Stock Alert
        </h3>
    """

    for product in dashboard_data['low_stock_products']:
        low_stock_html += f"""
        <div style="display: flex; justify-content: space-between; align-items: center; padding: 10px 0; border-bottom: 1px solid #f3f4f6;">
            <div>
                <span style="font-weight: 500; color: #374151;">{product['model']}</span>
                <div style="color: #6b7280; font-size: 14px;">{product['brand']}</div>
            </div>
            <span style="background: #fef3c7; color: #92400e; padding: 4px 8px; border-radius: 12px; font-size: 12px;">
                {product['stock_quantity']} left
            </span>
        </div>
        """

    low_stock_html += "</div>"

    # Top value inventory
    top_value_html = """
    <div style="background: white; border-radius: 8px; padding: 20px; border: 1px solid #e5e7eb;">
        <h3 style="color: #374151; margin-bottom: 15px;">💰 Top Value Inventory</h3>
    """

    for product in dashboard_data['top_value_inventory']:
        total_value = product['price'] * product['stock_quantity']
        top_value_html += f"""
        <div style="display: flex; justify-content: space-between; align-items: center; padding: 10px 0; border-bottom: 1px solid #f3f4f6;">
            <div>
                <span style="font-weight: 500; color: #374151;">{product['model']}</span>
                <div style="color: #6b7280; font-size: 14px;">{product['brand']} • {product['stock_quantity']} × ${product['price']}</div>
            </div>
            <span style="color: #10b981; font-weight: bold;">${total_value:,}</span>
        </div>
        """

    top_value_html += "</div>"

    return metrics_html + f"""
    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
        {low_stock_html}
        {top_value_html}
    </div>
    """

# Create Gradio interface
def create_interface():
    with gr.Blocks(
        title="Makers Tech ChatBot",
        theme=gr.themes.Base(  # Cambiar de Soft a Base
            primary_hue="blue",
            secondary_hue="gray",
            neutral_hue="gray",
        ).set(
            body_background_fill="white",
            body_text_color="black",
            block_background_fill="white",
            block_label_text_color="black"
        ),
            css="""
            .gradio-container {
                max-width: 1200px !important;
            }
            .tab-nav {
                background: #f8fafc;
                border-radius: 8px;
                padding: 4px;
            }
            """
         ) as demo:

            # Header
            gr.Markdown("""
            <div style="text-align: center; padding: 20px 0;">
                <div style="display: flex; align-items: center; justify-content: center; margin-bottom: 10px;">
                    <div style="background: #6366f1; color: white; padding: 8px; border-radius: 8px; margin-right: 10px; font-size: 20px;">🛍️</div>
                    <h1 style="margin: 0; color: black;">Makers Tech</h1>
            <p style="color: #333333; font-size: 16px; margin: 0;">
            </div>

            <div style="text-align: center; margin-bottom: 30px;">
                <h2 style="color: #374151; margin-bottom: 10px;">Welcome to Makers Tech ChatBot</h2>
                <p style="color: #6b7280;">Experience our AI-powered inventory assistant, smart recommendations, and real-time analytics dashboard</p>
            </div>
            """)

            # Tabs
            with gr.Tabs():
                # AI Chat Tab
                with gr.TabItem("🤖 AI Chat"):
                    gr.Markdown("### Makers Tech Assistant\n*AI-Powered Inventory Helper*")

                    with gr.Row():
                        with gr.Column(scale=2):
                            chatbot_interface = gr.Chatbot(
                                height=400,
                                show_label=False,
                                container=True,
                                type="messages"
                            )

                            with gr.Row():
                                msg = gr.Textbox(
                                    placeholder="Ask about our inventory, prices, or get recommendations...",
                                    container=False,
                                    scale=4
                                )
                                submit_btn = gr.Button("Send", variant="primary", scale=1)

                        with gr.Column(scale=1):
                            gr.Markdown("### Quick Stats")
                            dashboard_data = chatbot.get_dashboard_data()
                            gr.Markdown(f"""
                            **Total Products:** {dashboard_data['total_products']} 📦
                            **Computers:** {len([p for p in chatbot.inventory if p['category'] == 'Computers'])} 💻
                            **Phones:** {len([p for p in chatbot.inventory if p['category'] == 'Phones'])} 📱
                            **Accessories:** {len([p for p in chatbot.inventory if p['category'] == 'Accessories'])} 🎧
                            """)

                            gr.Markdown("### Try These Questions")
                            example_questions = [
                                "How many computers are available?",
                                "Tell me about the Dell laptop",
                                "What's the best phone under $1000?",
                                "Show me Apple products"
                            ]

                            for question in example_questions:
                                example_btn = gr.Button(question, size="sm")
                                example_btn.click(
                                    lambda q=question: (
                                        [{"role": "user", "content": q},
                                         {"role": "assistant", "content": chatbot.process_user_input(q)}],
                                        ""
                                    ),
                                    outputs=[chatbot_interface, msg]
                                )

                    # Chat functionality
                    msg.submit(chat_interface, [msg, chatbot_interface], [chatbot_interface, msg])
                    submit_btn.click(chat_interface, [msg, chatbot_interface], [chatbot_interface, msg])

                # Products Tab
                with gr.TabItem("📦 Products"):
                    gr.Markdown("### Product Catalog")
                    products_display = gr.HTML(get_products_display())

                # AI Recommendations Tab
                with gr.TabItem("✨ AI Recommendations"):
                    gr.Markdown("### Smart Product Recommendations")

                    with gr.Row():
                        with gr.Column():
                            gr.Markdown("#### Tell us what you're looking for:")
                            use_case = gr.Radio(
                                ["Gaming", "Work/Productivity", "General Use", "Creative Work"],
                                label="Primary Use Case",
                                value="General Use"
                            )
                            budget = gr.Slider(
                                minimum=100,
                                maximum=3000,
                                value=1000,
                                step=100,
                                label="Budget ($)"
                            )
                            brand_pref = gr.Dropdown(
                                ["Any", "Apple", "Dell", "HP", "Samsung", "Logitech"],
                                label="Brand Preference",
                                value="Any"
                            )

                            recommend_btn = gr.Button("Get Recommendations", variant="primary")

                        with gr.Column():
                            recommendations_output = gr.HTML("")

                    def get_recommendations(use_case, budget, brand_pref):
                        # Simple recommendation logic
                        filtered_products = chatbot.inventory.copy()

                        if brand_pref != "Any":
                            filtered_products = [p for p in filtered_products if p['brand'] == brand_pref]

                        filtered_products = [p for p in filtered_products if p['price'] <= budget]

                        if not filtered_products:
                            return "<p>No products match your criteria. Try adjusting your filters.</p>"

                        # Sort by relevance (simplified)
                        filtered_products.sort(key=lambda x: x['price'])

                        html = f"<h3>Recommendations for {use_case} (Budget: ${budget})</h3>"

                        for i, product in enumerate(filtered_products[:3]):
                            recommendation_level = ["🌟 Highly Recommended", "👍 Recommended", "💡 Consider"][min(i, 2)]
                            html += f"""
                            <div style="border: 1px solid #e5e7eb; border-radius: 8px; padding: 15px; margin: 10px 0; background: white;">
                                <div style="display: flex; align-items: center; margin-bottom: 10px;">
                                    <span style="font-size: 24px; margin-right: 10px;">{product['emoji']}</span>
                                    <div>
                                        <h4 style="margin: 0; color: #374151;">{recommendation_level}</h4>
                                        <h3 style="margin: 5px 0; color: #6366f1;">{product['brand']} {product['model']}</h3>
                                    </div>
                                </div>
                                <p style="color: #6b7280; margin: 10px 0;">Perfect for {use_case.lower()} with great performance and value.</p>
                                <div style="display: flex; justify-content: space-between; align-items: center;">
                                    <span style="font-size: 20px; font-weight: bold; color: #10b981;">${product['price']:,}</span>
                                    <span style="color: #6b7280;">{product['stock_quantity']} in stock</span>
                                </div>
                            </div>
                            """

                        return html

                    recommend_btn.click(
                        get_recommendations,
                        inputs=[use_case, budget, brand_pref],
                        outputs=recommendations_output
                    )

                # Analytics Tab
                with gr.TabItem("📊 Analytics"):
                    gr.Markdown("### Inventory Analytics Dashboard")

                    # Dashboard metrics
                    dashboard_html = gr.HTML(get_dashboard_display())

                    # Charts
                    with gr.Row():
                        stock_chart = gr.Plot(create_stock_chart())
                        distribution_chart = gr.Plot(create_stock_distribution_chart())

            # Footer
            gr.Markdown("""
            <div style="text-align: center; margin-top: 40px; padding: 20px; background: #f8fafc; border-radius: 8px;">
                <h3 style="color: #374151; margin-bottom: 5px;">Makers Tech ChatBot Demo</h3>
                <p style="color: #6b7280; margin: 0;">Showcasing AI-powered customer service, smart recommendations, and real-time analytics</p>
            </div>
            """)

        return demo

# Launch the interface
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=True,
        show_error=True
    )

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 1171)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a78509ded2a9bb7c8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
